<a href="https://colab.research.google.com/github/ysjbluemoon/Applied-Deep-Learning/blob/master/Assignment%203/a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A3: TensorBoard

## About

In this assignment, you will design and run experiments to evaluate the impact of a few common parameters (like the choice of activation function, optimizer, and weight initialization strategy) and visualize the results in TensorBoard.

The starter code below shows the mechanics of using TensorBoard in Colab. Unlike the previous assignments, a limited amount of starter code is provided.  3c is an extra credit question, it's optional (you can receive full credit on this assignment without submitting it).

## Questions

### 3a. 
**Implement ReLU and compare against a previous activation function**.

The year is 2010. It is not commonly known that ReLU is a useful alternative to activation functions like Sigmoid or Tanh (nor has ReLU been implemented in the library you're using). Create a DNN to classify MNIST, and provide your own implementation of ReLU (instead of using a built-in method). Design and run an experiment to compare ReLU against other methods, and use TensorBoard to display your results. What differences do you observe, and why?

### 3b. 

**Optimizer and initalizer and soup**.

Do optimizers like Momentum or Adam really make a difference? How about different weight initialize strategies (like random normal, or glorot uniform?) Design and run experiments to find out, and use TensorBoard to display your results. What differences do you observe, and why?

### 3c. Extra credit (optional)

**Demonstrate the vanishing gradient problem**. 

Implement an especially deep neural network and train it on a simple dataset like MNIST. Choose activation functions, initialization strategies, and an optimizer that are likely to cause this behavior. Produce histograms of activations and gradients at various layers during training. What do you see? Next, adjust the parameters above to correct this behavior. Visualize and compare the results.

## Submission instructions

Please submit your assignment on CourseWorks by uploading a zip file that includes:

* A Jupyter notebook, containing complete code to reproduce your experiments, and saved output showing your results.

* A README file (plaintext is fine). This should contain your written conclusions for each question. These can be brief (a couple paragraphs). Try to be specific in your answers (if ReLU outperfoms sigmoid, try to answer why).

* Plots / diagrams (.jpgs). Since it is not convenient to save TensorBoard diagrams directly in a Jupyter notebook, you can take screenshots of your plots and submit them along with your Jupyter notebook in a zip file on CourseWorks. Please name your diagrams appropriately, and refer to them names in your notebook.

If you are working in Colab, you can prepare your notebook for submission by ensuring that runs end-to-end, then saving and downloading it:

1. ```Runtime -> Restart and run all```
1. ```File -> Save```
1. ```File -> Download.ipynb```

## Starter code for TensorBoard

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [0]:
%load_ext tensorboard 

**Caution**. The following cell will clear the logs directory. If you're running this on your local machine, be careful executing it.

In [0]:
!rm -rf ./tensorboard-logs/ # Clear any logs from previous runs

Import a dataset

In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train, x_test = x_train / 255.0, x_test / 255.0

## First style

Define a simple model

In [0]:
from tensorflow.keras.layers import Dense, Flatten

def create_model():
  model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='softmax'),
  ])
  return model

Create a logs directory

In [0]:
import datetime 
import os
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

### Run an experiment

In [0]:
from tensorflow.keras.optimizers import SGD

model = create_model() 
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False) 
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "exp1")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=1, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

### Run a second experiment

In [0]:
model = create_model() 
opt = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "exp2")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

### Start TensorBoard and compare the results

In [0]:
%tensorboard --logdir "$log_dir"

## Second style
Using a Subclassed model and a GradientTape

Prepre the dataset

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_ds = train_ds.shuffle(60000).batch(32)
test_ds = test_ds.batch(32)

Define a simple model

In [0]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten(input_shape=(28, 28))
    self.d1 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    return self.d1(x)

model = MyModel()

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')

Training and testing routines

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_fn(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

Prepare log writers (previously, these were handled by the callback)

In [0]:
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

train_writer = tf.summary.create_file_writer(os.path.join(log_dir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(log_dir, "test"))

Train and log summaries

In [0]:
EPOCHS = 2

for epoch in range(EPOCHS):
  
  for images, labels in train_ds:
    train_step(images, labels)
    
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  
  with train_writer.as_default():
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    tf.summary.scalar('loss', train_loss.result(), step=epoch)

    # ====
    # Demo: show how to use histogram summaries
    # Create and log some random data
    # Useful if you're attemping the extra credit question
    # ====
    data = tf.random.normal((32, 100))
    tf.summary.histogram('random', 
                         data,
                         step=epoch, 
                         description='Your description')
    
  with test_writer.as_default():
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    
  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

In [0]:
%tensorboard --logdir "$log_dir"